## Задание

1. Мы будем работать с (частичными) данными lenta.ru отсюда: https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации (в качестве метки класса будем использовать поле topic). В качестве данных для классификации в пунктах 3 и 5 возьмите
    - только заголовки (title)
    - только тексты новости (text)
    - и то, и другое
3. Обучите fastText для классификации текстов по темам. Сравните качество для разных данных из п. 2.
4. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста / заголовка / текста+заголовка как среднего вектора входящих в него слов. 
     - (Бонус) Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.
5. Обучите на полученных средних векторах алгоритм классификации, сравните полученное качество с классификатором fastText. 

In [4]:
!wget -O data/lenta-ru-news-part.csv https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0

--2024-12-23 10:17:15--  https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0
Resolving proxy-ws.cbank.kz (proxy-ws.cbank.kz)... 10.1.246.2
Connecting to proxy-ws.cbank.kz (proxy-ws.cbank.kz)|10.1.246.2|:8080... connected.
  Self-signed certificate encountered.
Proxy request sent, awaiting response... 403 Forbidden
2024-12-23 10:17:15 ERROR 403: Forbidden.



In [ ]:
lenta = pd.read_csv('data/lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
lenta.head()